In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# girder_client docs are here: https://girder.readthedocs.io/en/stable/python-client.html#the-python-client-library
import girder_client

from dotenv import load_dotenv

import metadata_pipeline as mph 
import brain_region_maps as brm

SyntaxError: invalid syntax (brain_region_maps.py, line 1716)

In [ ]:
load_dotenv()
gc = girder_client.GirderClient(apiUrl=os.environ.get("dsaApiUrl"))
gc.authenticate(apiKey=os.environ.get("dsaApiToken"))

In [ ]:
NeuroPathDemoImages_Collection='63d98235e08050e0a7a8d75d'

In [ ]:
mph.populateMetadata(gc, collectionID=NeuroPathDemoImages_Collection, outputFailed=True)

In [ ]:
mph.auditMetadata(gc, collectionID=NeuroPathDemoImages_Collection, outputRecords=True)

In [ ]:
mph.getSummaryStats(gc, collectionID=NeuroPathDemoImages_Collection)

In [ ]:
mph.getMissingBrainRegion()